# Crime Detection Model Generator

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import joblib

In [2]:
# Load and preprocess data
df = pd.read_csv('crime_data.csv')
df.columns = df.columns.str.strip()

df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=False)
df['hour'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce').dt.hour.fillna(0).astype(int)
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['weekday'] = df['Date'].dt.weekday

In [ ]:
~# Encode categorical variables
le_location = LabelEncoder()
df['Location_encoded'] = le_location.fit_transform(df['Locality_Name'])
joblib.dump(le_location, 'location_encoder.pkl')

le_crime = LabelEncoder()
df['Crime_encoded'] = le_crime.fit_transform(df['Crime_Type'])
joblib.dump(le_crime, 'crime_encoder.pkl')

['crime_encoder.pkl']

In [4]:
# Feature selection and scaling
X = df[['Location_encoded', 'Crime_encoded', 'month', 'day', 'weekday', 'hour']].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.pkl')
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

In [5]:
# Build Conv1D + BiLSTM Autoencoder
input_layer = tf.keras.Input(shape=(6, 1))
x = tf.keras.layers.Conv1D(32, kernel_size=2, padding='same', activation='relu')(input_layer)
x = tf.keras.layers.MaxPooling1D(pool_size=2, padding='same')(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.UpSampling1D(size=2)(x)
x = tf.keras.layers.Conv1D(1, kernel_size=2, padding='same', activation='sigmoid')(x)

autoencoder = tf.keras.Model(inputs=input_layer, outputs=x)
from tensorflow.keras.losses import MeanSquaredError
autoencoder.compile(optimizer='adam', loss=MeanSquaredError())

In [6]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=32, verbose=1, callbacks=[early_stop])

Epoch 1/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.0347
Epoch 2/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0013
Epoch 3/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.0011
Epoch 4/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 9.4074e-04
Epoch 5/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 7.7584e-04
Epoch 6/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 6.4035e-04
Epoch 7/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 5.4974e-04
Epoch 8/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 5.0298e-04
Epoch 9/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 4.4934e-04
Epoch 10/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 3.9355e-04
Epoch 11/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 3.5334e-04
Epoch 12/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 3.1116e-04
Epoch 13/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 2.8877e-04
Epoch 14/30
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - l

In [7]:
# Save model
autoencoder.save('conv_bilstm_autoencoder.h5')

In [8]:
# Predict and calculate reconstruction error
reconstructed = autoencoder.predict(X_scaled, verbose=0)
mse = np.mean(np.power(X_scaled - reconstructed, 2), axis=(1, 2))
df['reconstruction_error'] = mse

In [9]:
# Identify anomalies
threshold = np.percentile(mse, 95)
anomalies = df[df['reconstruction_error'] > threshold]
print("\nTop anomalies based on reconstruction error:")
print(anomalies[['Locality_Name', 'Crime_Type', 'Date', 'Time', 'reconstruction_error']].head())


Top anomalies based on reconstruction error:
   Locality_Name           Crime_Type                Date              Time  \
5      Falaknuma              ASSAULT 2020-01-01 05:00:00  01-01-2020 17:09   
6      Charminar     VEHICLE - STOLEN 2020-01-01 06:00:00  01-01-2020 14:08   
16    gachibowli         DRUG OFFENSE 2020-01-01 16:00:00  02-01-2020 02:57   
17    Patancheru                ARSON 2020-01-01 17:00:00  01-01-2020 23:09   
52    gachibowli  PUBLIC INTOXICATION 2020-01-03 04:00:00  03-01-2020 22:01   

    reconstruction_error  
5               0.000255  
6               0.000255  
16              0.000743  
17              0.000306  
52              0.000464  
